<a href="https://www.kaggle.com/code/sofiamatias/learning-equality-model-sm?scriptVersionId=117411713" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Kaggle Challenge - Learning Equality

https://www.kaggle.com/competitions/learning-equality-curriculum-recommendations/overview

## Goal of the Competition

The goal of this competition is to streamline the process of matching educational content to specific topics in a curriculum. You will develop an accurate and efficient model trained on a library of K-12 educational materials that have been organized into a variety of topic taxonomies. These materials are in diverse languages, and cover a wide range of topics, particularly in STEM (Science, Technology, Engineering, and Mathematics).

Your work will enable students and educators to more readily access relevant educational content to support and supplement learning.

## Submission File

For each **topic_id** in the test set, you must predict a space-delimited list of recommended **content_ids** for that topic. The file should contain a header and have the following format:

~~~
topic_id,content_ids
t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c_76231f9d0b5e
t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c_ebb7fdf10a7e
t_00069b63a70a,c_11a1dc0bfb99
...
~~~

## Scoring
Mean F2 score

## Evaluation Metric - Efficiency Scoring
We compute a submission's efficiency score by:

\begin{equation} \text{Efficiency} = \frac{1}{ \text{Benchmark} - \max\text{F2} }\text{F2} + \frac{1}{32400}\text{RuntimeSeconds} \end{equation}


where **F2** is the submission's score on the main competition metric, **Benchmark** is the score of the benchmark sample_submission.csv, **maxF2** is the maximum  of all submissions on the Private Leaderboard, and **RuntimeSeconds** is the number of seconds it takes for the submission to be evaluated. The objective is to minimize the efficiency score.

During the training period of the competition, you may see a leaderboard for the public test data in the following notebook, updated daily: Efficiency Leaderboard. After the competition ends, we will update this leaderboard with efficiency scores on the private data. During the training period, this leaderboard will show only the rank of each team, but not the complete score.

# How To Solve This Challenge

## Model Train

* Calculate embeddings: for topics (title, description) and contents (title, description and text), use SentenceTransformer. Split sentences by language.
* Use KNN model: train model with content embeddings and use topic embeddings to predict content matches, use k=10, 20, 30, 50
* Calculate F2 to choose best k.
* Submit predictions
* Set X_train, X_test, y_train, y_test: split by category (use 'aligned' for validation), set has_content = True for validation. Check dimensions
* Use all features for X, including topic_id and content_id, topic_title and content_title. Use the KNN predictions to "mount" X and y. Check dimensions.
* Use correlations to get y: if KNN topic-content match with correlations, y is 1, else is 0
* Use SVM RBF or KNN again for multiclass classification
* Use RandomizedSearch to get best hyperparameters. Use model score "recall". Split the dataset in languages for training: train model for each language individually
* Calculate F2 and see if we've got improvements from KNN.
* Submit predictions

## Submissions

* Use submission sample and get topics and contents
* Apply KNN model to get topic-content matches. Get contents per topic
* Filter "good matches" with second classification model
* Compare predicts with submission sample and calculate F2

## Efficiency

* Use KNN and 2nd model without GPU and check if it takes a long time to calculate sample submissions

# Data

## Imports

In [1]:
drive_path = '/kaggle/input/learning-equality-curriculum-recommendations/'
dataset_path = '/kaggle/input/learningequalityfiles/'
work_path = '/kaggle/working/'
model_path = '/kaggle/input/sentence-transformer-package/ST-all-MiniLM-L6-v2-trained/ST-all-MiniLM-L6-v2-trained'

In [2]:
import sys, os
sys.path.append("../input/sentence-transformer-package/sentence-transformers-2.2.2/sentence-transformers-2.2.2") 
import sentence_transformers

In [3]:
import pandas as pd
import numpy as np
import string
import torch
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics import precision_score, recall_score, fbeta_score
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_validate

# Data Collection

## Load dataframes

In [4]:
# load 'topics' data into pandas dataframe
#df_topics = pd.read_csv(f'{drive_path}topics.csv', index_col=0).fillna({"title": "", "description": ""})
#print (f"\nLoaded 'df_topics'")
#df_topics

In [5]:
# load 'content' data into pandas dataframe
#df_content = pd.read_csv(f'{drive_path}content.csv', index_col=0).fillna("")
#print (f"\nLoaded 'df_content'")
#df_content

In [6]:
# load 'topic_breadcrumbs' data into pandas dataframe
#df_topic_breadcrumbs = pd.read_csv(f'{dataset_path}topic_breadcrumbs.csv', index_col=0)
#print (f"\nLoaded 'df_topic_breadcrumbs'")
#df_topic_breadcrumbs

## Load test dataframes

In [7]:
# load 'topics.csv' and 'content.csv' with sample submission data
# This will allow to simulate submitting real predicts for the challenge
submit_mode=False
if (os.path.exists (f'{drive_path}topics.csv')) and (submit_mode):
    df_topics = pd.read_csv(f'{drive_path}topics.csv', index_col=0).fillna({"title": "", "description": ""})
else:
    df_topics = pd.read_csv(f'{dataset_path}topics.csv', index_col=0).fillna({"title": "", "description": ""})
if (os.path.exists (f'{drive_path}content.csv')) and (submit_mode):
    df_content = pd.read_csv(f'{drive_path}content.csv', index_col=0).fillna({"title": "", "description": ""})
else:
    df_content = pd.read_csv(f'{dataset_path}content.csv', index_col=0).fillna({"title": "", "description": ""})
if (os.path.exists (f'{drive_path}correlations.csv')) and (submit_mode):
    df_corr = pd.read_csv(f'{drive_path}correlations.csv', index_col=0)
else:
    df_corr = pd.read_csv(f'{dataset_path}correlations.csv', index_col=0)  
display(df_topics.head())
display(df_content.head())
display(df_corr.head())

,title,description,channel,category,level,language,parent,has_content
id,,,,,,,,
t_00004da3a1b2,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True
t_00068291e9a4,Entradas e saídas de uma função,Entenda um pouco mais sobre funções.,8e286a,source,4,pt,t_d14b6c2a2b70,True
t_00069b63a70a,Transcripts,,6e3ba4,source,3,en,t_4054df11a74e,True
t_0006d41a73a8,Графики на експоненциални функции (Алгебра 2 н...,Научи повече за графиките на сложните показате...,000cf7,source,4,bg,t_e2452e21d252,True
t_4054df11a74e,Flow Charts: Logical Thinking?,This lesson is focused on flow charts. It supp...,6e3ba4,source,2,en,t_acbbd893e6af,True


,title,description,kind,text,language,copyright_holder,license
id,,,,,,,
c_0c6473c3480d,Чертане на показателни финкции,"Сал чертае y=-2*3ˣ+5, като използва интерактив...",video,"Казват ни: ""Използвай интерактивната графика п...",bg,Khan Academy,CC BY-NC-SA
c_1108dd0c7a5d,Молив като резистор,"Моливът причинява промяна в отклонението, подо...",video,NaN,bg,NaN,NaN
c_11a1dc0bfb99,Flow Charts: Logical..: Written Transcript of ...,"Written Transcript of this video lesson, Engli...",document,Español\t\n \nTítulo\t\n de\t\n la\t\n lec...,en,MIT Blossoms,CC BY-NC-SA
c_1c57a1316568,Графики на показателни функции (стар пример),Сал свързва следните четири функции към подход...,video,"Тук имаме четири графики и 4 израза, задаващи ...",bg,Khan Academy,CC BY-NC-SA
c_3695c5dc1df6,Additional Resources for Flow Charts: Logical ...,Additional resources and links.,html5,This MIT BLOSSOMS video lesson focuses on the ...,en,MIT Blossoms,CC BY-NC-SA


,content_ids
topic_id,
t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...
t_00069b63a70a,c_11a1dc0bfb99
t_0006d41a73a8,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...
t_0008768bdee6,c_34e1424229b4 c_7d1a964d66d5 c_aab93ee667f4


In [8]:
# load 'correlations' data into pandas dataframe
df_corr = pd.read_csv(f'{drive_path}correlations.csv', index_col=0)
print (f"\nLoaded 'df_corr'")
# Restructure 'df_corr' (correlations): explode the target column to be more practical to use - one topic -> one content
y = df_corr.copy()
y['content_ids'] = y.content_ids.str.split(' ')
y = y.explode('content_ids')
y.reset_index(inplace=True)
print (f"\nChanged 'df_corr' to exploded 'y'")
display(y)


Loaded 'df_corr'

Changed 'df_corr' to exploded 'y'


,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d
1,t_00004da3a1b2,c_376c5a8eb028
2,t_00004da3a1b2,c_5bc0e1e2cba0
3,t_00004da3a1b2,c_76231f9d0b5e
4,t_00068291e9a4,c_639ea2ef9c95
...,...,...
279914,t_fff9e5407d13,c_d64037a72376
279915,t_fffbe1d5d43c,c_46f852a49c08
279916,t_fffbe1d5d43c,c_6659207b25d5
279917,t_fffe14f1be1e,c_cece166bad6a


In [9]:
# load 'sample_submission' data into pandas dataframe
#df_sample_subs = pd.read_csv(f'{drive_path}sample_submission.csv', index_col=0)
#df_sample_subs
# Get features data from existing df's
#sample_subs = df_sample_subs.copy()
#sample_subs['content_ids'] = sample_subs.content_ids.str.split(' ')
#sample_subs = sample_subs.explode('content_ids')
#df_subs_topics = df_topics.reset_index().merge (df_sample_subs.reset_index(), how='inner', left_on='id', right_on='topic_id').fillna('')
#df_subs_topics = df_subs_topics.set_index('id').drop(columns=['topic_id', 'content_ids']).fillna('')
#df_subs_contents = df_content.reset_index().merge (sample_subs, how='inner', left_on='id', right_on='content_ids')
#df_subs_contents = df_subs_contents.set_index('id').drop(columns=['content_ids']).fillna('')
#df_subs_topics.to_csv(f'{work_path}topics.csv')
#df_subs_contents.to_csv(f'{work_path}content.csv')

## Create Topic Breadcrumbs

In [10]:
df_topic = df_topics.copy()
for i, topic in enumerate(df_topic.index):
    print (df_topic.loc[topic].parent in df_topic.index)

False
False
True
False
False


In [11]:
# define some helper functions and classes to aid with data traversal

def get_topic_breadcrumbs (df_topic):
    print (f"Creating breadcrumbs...")
    topic_breadcrumbs=[]
    topic_ids=[]
    if 'topic_id' == df_topics.index.name:
        col_name = 'topic_id'
    else:
        col_name = 'id'
    for i, topic in enumerate(df_topic.index):
        if (i % 5000 == 0) and (i > 0):
            print (f'Created {i} breadcrumbs...')
        if df_topic.loc[topic].parent in df_topic.index:
            topic_ids.append (topic)
            topic_breadcrumbs.append (Topic(topic).get_breadcrumbs(separator=' '))
    print (f"Creating dataframe from breadcrumbs 'df_topics_breadcrumbs'...")
    df_topics_breadcrumbs = pd.DataFrame (topic_breadcrumbs, 
                                          index = topic_ids,
                                          columns = ['topic_breadcrumbs'])
    return df_topics_breadcrumbs


def print_markdown(md):
    display(Markdown(md))

class Topic:
    def __init__(self, topic_id):
        self.id = topic_id

    @property
    def parent(self):
        parent_id = df_topics.loc[self.id].parent
        if pd.isna(parent_id):
            return None
        else:
            return Topic(parent_id)

    @property
    def ancestors(self):
        ancestors = []
        parent = self.parent
        while (parent is not None) and (parent.id in df_topics.index):
            ancestors.append(parent)
            parent = parent.parent
        return ancestors

    @property
    def siblings(self):
        if not self.parent:
            return []
        else:
            return [topic for topic in self.parent.children if topic != self]

    @property
    def content(self):
        if self.id in df_corr.index:
            return [ContentItem(content_id) for content_id in df_corr.loc[self.id].content_ids.split()]
        else:
            return tuple([]) if self.has_content else []

    def get_breadcrumbs(self, separator=" >> ", include_self=True, include_root=True):
        ancestors = self.ancestors
        if include_self:
            ancestors = [self] + ancestors
        if not include_root:
            ancestors = ancestors[:-1]
        return separator.join(reversed([a.title for a in ancestors]))

    @property
    def children(self):
        return [Topic(child_id) for child_id in df_topics[df_topics.parent == self.id].index]

    def subtree_markdown(self, depth=0):
        markdown = "  " * depth + "- " + self.title + "\n"
        for child in self.children:
            markdown += child.subtree_markdown(depth=depth + 1)
        for content in self.content:
            markdown += ("  " * (depth + 1) + "- " + "[" + content.kind.title() + "] " + content.title) + "\n"
        return markdown

    def __eq__(self, other):
        if not isinstance(other, Topic):
            return False
        return self.id == other.id

    def __getattr__(self, name):
        return df_topics.loc[self.id][name]

    def __str__(self):
        return self.title
    
    def __repr__(self):
        return f"<Topic(id={self.id}, title=\"{self.title}\")>"


class ContentItem:
    def __init__(self, content_id):
        self.id = content_id

    @property
    def topics(self):
        return [Topic(topic_id) for topic_id in df_topics.loc[df_corr[df_corr.content_ids.str.contains(self.id)].index].index]

    def __getattr__(self, name):
        return content_df.loc[self.id][name]

    def __str__(self):
        return self.title
    
    def __repr__(self):
        return f"<ContentItem(id={self.id}, title=\"{self.title}\")>"

    def __eq__(self, other):
        if not isinstance(other, ContentItem):
            return False
        return self.id == other.id

    def get_all_breadcrumbs(self, separator=" >> ", include_root=True):
        breadcrumbs = []
        for topic in self.topics:
            new_breadcrumb = topic.get_breadcrumbs(separator=separator, include_root=include_root)
            if new_breadcrumb:
                new_breadcrumb = new_breadcrumb + separator + self.title
            else:
                new_breadcrumb = self.title
            breadcrumbs.append(new_breadcrumb)
        return breadcrumbs

## Data Cleaning

## Data cleaning params

In [12]:
levels = {1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five', 6: 'six', 7: 'seven', 8: 'eight',
          9: 'nine', 10: 'ten', 0: 'zero'}
topics_cols = ['title', 'description', 'topic_breadcrumbs']
content_cols = ['title', 'description', 'text']
cat_for_val = 'aligned'
print (f"\nLoaded cleaning parameters")


Loaded cleaning parameters


## Data cleaning functions

In [13]:
# Clean up text
def clean_text(text_col):
    """
    Clean ponctuation and special chars from a dataframe column
    """
    punctuations = string.punctuation
    text_col = text_col.str.replace('\W', ' ', regex=True)
    for punct in string.punctuation:
        text_col = text_col.str.replace(punct, ' ', regex=True)
    return text_col

## Topics

*   Changing 'level' values from integers to strings
*   Clean strings: ponctuation and special chars (\n\t...)
*   Sort values by language

In [14]:
def build_topic_features (topics_df, df_topics_breadcrumbs, levels, topic_cols):
    """
    Create 'topics_features' from df_topics, clean parameters and functions
    """
    print (f"\nCreating and cleaning topic features...")
    topics_features = topics_df.copy()
    topics_features = topics_features.replace ({'level': levels})
    topics_features = topics_features.merge (df_topic_breadcrumbs, how='outer', right_index=True, left_index=True)
    for col in topics_cols:
        topics_features[col] = clean_text(topics_features[col])
    topics_features['sentences'] = topics_features[topics_cols].apply(lambda x: '.'.join(x.dropna().astype(str)), axis=1)
    topics_features = topics_features.drop(columns=['parent'] + topics_cols) 
    print (f"\nCreated 'topic_features'")
    display (topics_features.head())
    
    return topics_features


## Contents


*   Remove columns 'copyright_holder' and 'language' (for 'language' assume that topic-content match with correlations is same language)
*   Clean strings: ponctuation and special chars (\n\t...)

In [15]:
def build_content_features (content_df, content_cols):
    """
    Create 'content_features' from df_content, clean parameters and functions
    """
    print (f"\nCreating and cleaning content features...")
    content_features = content_df.copy()
    for col in content_cols:
        content_features[col] = clean_text(content_features[col])
    content_features['sentences'] =  content_features[content_cols].apply(lambda x: '.'.join(x.dropna().astype(str)), axis=1)
    content_features = content_features.drop(columns=['copyright_holder'] + content_cols)
    print (f"\nCreated 'content_features'")
    display (content_features.head())
    
    return content_features

# Model training

### Models params

In [16]:
model_path

'/kaggle/input/sentence-transformer-package/ST-all-MiniLM-L6-v2-trained/ST-all-MiniLM-L6-v2-trained'

In [17]:
#General
device = 'cpu'
if torch.cuda.is_available():
    device = torch.device('cuda:0')
#Sentence Transformer
all_mini = 'all-MiniLM-L6-v2'
paraphrase = 'paraphrase-multilingual-MiniLM-L12-v2'
trained_model = model_path
# KNN
k = [5,10,20,30,50]
alg = 'kd_tree' #'ball_tree','brute'
leaf = 10
print (f"\nLoaded models parameters")


Loaded models parameters


### Scoring functions

In [18]:
def model_scoring (y_test, y_pred, average='macro'):
    """
    Calculate precision, recall and f2-score for y_test and y_pred
    """
    precision = precision_score(y_test['content_ids'], y_pred['content_ids'], average=average)
    print ('Precision:', precision)
    recall = recall_score(y_test['content_ids'], y_pred['content_ids'], average=average)
    print ('Recall:', recall)
    F2macro = fbeta_score(y_test['content_ids'], y_pred['content_ids'], beta=2, average='macro')
    print ('F2 macro:', F2macro)
    F2micro = fbeta_score(y_test['content_ids'], y_pred['content_ids'], beta=2, average='micro')
    print ('F2 micro:', F2micro)

In [19]:
def calculate_Fscore(prediction_df, actual_df):
    
    prediction_df['content_ids'] = prediction_df.content_ids.str.split(' ')
    prediction_df.columns=['topic_id', 'content_ids_pred']
    actual_df['content_ids'] = actual_df.content_ids.str.split(' ')
    actual_df.columns=['topic_id', 'content_ids_actual']
    df = pd.merge(prediction_df, actual_df, how='inner', on='topic_id')
    df['correct_pred'] = df[['content_ids_pred', 'content_ids_actual']].apply(lambda x: len([d for d in x[0] if d in x[1]]), axis=1)
    df['precision'] = df['correct_pred']/(df.content_ids_actual.str.len() + 1e-7)
    df['recall'] = df['correct_pred']/(df.content_ids_pred.str.len() + 1e-7)
    for beta in [0.5, 1, 2]:
        df['f'+str(beta)] = ((1 + beta**2) * df['precision'] * df['recall'])/((beta**2 * df['precision']) + df['recall'] + 1e-7) 
    
    return df

In [20]:
example_pred = [{ 'topic_id': 't1', 'content_ids': 'c1 c2 c3 c10 c12 c13 c14'}, { 'topic_id': 't2', 'content_ids': 'c1 c2 c3 c14'}]
example_actual = [{ 'topic_id': 't1', 'content_ids': 'c1 c2 c3 c4 c5'}, { 'topic_id': 't2', 'content_ids': 'c2 c4 c5'}]
calculate_Fscore(pd.DataFrame(example_pred), pd.DataFrame(example_actual))

,topic_id,content_ids_pred,content_ids_actual,correct_pred,precision,recall,f0.5,f1,f2
0,t1,"[c1, c2, c3, c10, c12, c13, c14]","[c1, c2, c3, c4, c5]",3,0.600000,0.428571,0.555555,0.500000,0.454545
1,t2,"[c1, c2, c3, c14]","[c2, c4, c5]",1,0.333333,0.250000,0.312500,0.285714,0.263158


## Get embeddings

* Use output as pytorch tensor and normalize tensor embeddings
* Convert to numpy array

In [21]:
def get_embeddings(topics_features, content_features, langs, langs_limit):
    """
    Calculate embeddings for both topics and content
    From text in columns:
    topic_cols=['title','description', 'topic_breadcrumbs'] 
    content_cols=['title','description', 'text']
    """
    device = 'cpu'
    if torch.cuda.is_available():
        device = torch.device('cuda:0')

    encoder = SentenceTransformer(trained_model)
    topics_id_for_embeddings=[]
    topics_embeddings=[]
    content_ids_for_embeddings=[]
    content_embeddings=[]
    topics_aux = []
    content_aux = []

    for i, lang in enumerate(langs[:langs_limit]):
        print (f'\nGetting embeddings for {i+1}:{lang} from {langs_limit} languages')
        # topics embeddings
        topics_sentences = topics_features[topics_features.language == lang]['sentences']
        topics_aux = encoder.encode(topics_sentences, convert_to_tensor=True, show_progress_bar=False)
        topics_id_for_embeddings.append (topics_sentences.index)
        topics_embeddings.append (util.normalize_embeddings(topics_aux.to(device)).detach().cpu().numpy())
        # content embeddings
        content_sentences = content_features[content_features.language == lang]['sentences']
        if len(content_sentences) == 0:
            content_embeddings.append (np.array([0]).reshape(1,-1)) 
            content_ids_for_embeddings.append (content_sentences.index)
            continue
        else:
            content_ids_for_embeddings.append (content_sentences.index)
            content_aux = encoder.encode(content_sentences, convert_to_tensor=True, show_progress_bar=True)
            content_embeddings.append (util.normalize_embeddings(content_aux.to(device)).detach().cpu().numpy())
    
    return topics_embeddings, content_embeddings, topics_id_for_embeddings, content_ids_for_embeddings

## Apply Nearest Neighbours

* Use unsupervised NN. Use GPU when available, otherwise use sklearn NN

### topic-content predictions functions (df's)

In [22]:
def get_preds(list_aux, topics_id_for_embeddings):
    """
    df_preds = Dataframe with topic_id's and KNN chosen content_ids
    """
    df = pd.DataFrame(list_aux)
    df['topic_id'] = topics_id_for_embeddings
    df = df.explode(['content_ids','topic_id'])
    df.content_ids = df.content_ids.str.join(' ')
    df = df.iloc[:,[0,2,1]].reset_index().drop(columns=['language', 'index'])
    df = df.fillna('')
    return df

In [23]:
def get_true(df_corr, df):
    """
    Ground truth
    df_true = Dataframe with same topic_id's from 'get_preds' function but content_id's taken from correlations, 
    for score calculations,
    """
    df_true = df_corr.copy().reset_index().merge (df, how='right', on='topic_id')[['topic_id', 'content_ids_x']].rename(columns={'content_ids_x': 'content_ids'})
    df_true = df_true.fillna('')
    return df_true

### Calculate k neighbors for contents per topic and F2 score

In [24]:
def k_neighbors (k, topics_embeddings, content_embeddings, topics_id_for_embeddings, content_ids_for_embeddings, df_corr, langs, lang_lim):
    """
    Calculate k neighbors of contents:id's for each topic_id
    """
    device = 'cpu'
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    if device == 'cpu':
        from sklearn.neighbors import NearestNeighbors
    else:
        import cudf
        import cuml
        from cuml.neighbors import NearestNeighbors

    print ('\nSetting up KNN model...')
    # Model
    min_sample = min(len(content_embeddings[i]) for i in range(len(content_embeddings)))
    if k > min_sample:
        k = min_sample
    print ('\n K value:', k)
    model = NearestNeighbors(n_neighbors=k, 
                             n_jobs=-1)

    print ('\nFitting KNN model...')
    # Fit and Predictions
    list_aux=[]
    topics_count = 0
    for lang in range(lang_lim):
        print (f'\nGetting content_ids for topics in {lang+1}:{langs[lang]} from {lang_lim} languages')
        topics_count = topics_count + len(topics_embeddings[lang])
        if content_embeddings[lang].size != 1:
            nbrs = model.fit(content_embeddings[lang])
            _, indices = nbrs.kneighbors(topics_embeddings[lang])
            content_id_list = [content_ids_for_embeddings[1][val] for i, val in enumerate (indices)]
            list_aux.append (dict (language=langs[lang], 
                                   content_ids=content_id_list))
        else:
            list_aux.append (dict (language=langs[lang], content_ids=np.zeros(len(topics_id_for_embeddings[lang]))))

    print ('\nCalculating scores...')
    #Define dataframes for predictions and ground truth
    df_preds = get_preds(list_aux, topics_id_for_embeddings)
    df_true = get_true(df_corr, df_preds)
    fscore = calculate_Fscore(df_preds.copy(), df_true.copy())
    print ('\nTopics to match content:', topics_count)
    print ('\nCorrect predictions:', fscore.correct_pred.sum())
    print ('\nF2 Score:', fscore.f2.mean())
    #print ('\n\nF2 score by sklearn functions:')
    #model_scoring (df_preds.copy(), df_true.copy(), average='micro')
    print ('\n')
    
    return model, df_preds, df_true, fscore

## K parameter vs F2 score

In [25]:
"""
Encoder = 'paraphrase'
Language_limit = 5
K0 = 5
F2_score0 = 0.14187983285626535
K1 = 10
F2_score1 = 0.14168308552036216
K2 = 20
F2_score2 = 0.13348882325195702
K3 = 30
F2_score3 = 0.1252348976238163
Language_limit = 20
K0 = 5
F2_score0 = 0.060680956642008434
k1 = 10
F2_score0 = 0.04720607798794654
k2 = 20
F2_score1 = 0.0348693757018806
k5 = 100
F2_score5 = 0.016168764671525976
Encoder = trained_model
Language_limit = 10
k1 = 2 #limited by the sample dataset
F2_score1 = 0.32051279617850204
"""


"\nEncoder = 'paraphrase'\nLanguage_limit = 5\nK0 = 5\nF2_score0 = 0.14187983285626535\nK1 = 10\nF2_score1 = 0.14168308552036216\nK2 = 20\nF2_score2 = 0.13348882325195702\nK3 = 30\nF2_score3 = 0.1252348976238163\nLanguage_limit = 20\nK0 = 5\nF2_score0 = 0.060680956642008434\nk1 = 10\nF2_score0 = 0.04720607798794654\nk2 = 20\nF2_score1 = 0.0348693757018806\nk5 = 100\nF2_score5 = 0.016168764671525976\nEncoder = trained_model\nLanguage_limit = 10\nk1 = 2 #limited by the sample dataset\nF2_score1 = 0.32051279617850204\n"

# Submissions



In [26]:
#Getting parameters
langs = df_topics['language'].value_counts (ascending=True).reset_index()['index'].to_list()
langs_limit = len(langs)
K = k[1]

#Getting topics breadcrumbs or topics context
print ("\n* Getting 'topics_breadcrumbs' dataframe...\n")
df_topic_breadcrumbs = get_topic_breadcrumbs (df_topics)

#Cleaning data and getting topics and contents features
print ('\n* Calculating topics_features dataframe...\n')
sub_topics_features = build_topic_features (df_topics, df_topic_breadcrumbs, levels, topics_cols)

print ('\n* Calculating content_features dataframe...\n')
sub_contents_features = build_content_features (df_content, content_cols)

#Calculating embeddings from text
print ('\n* Calculating embeddings for both topics and contents text columns...\n')
sub_topics_embeds, sub_contents_embeds, topic_id_embeds, content_ids_embeds = get_embeddings(sub_topics_features, sub_contents_features, langs, langs_limit)

#Calculating unsupervised k neighbours contents, for topics
print ('\n* Calculating k neighnors predictions...\n')
model, preds, ground_truth, fscore = k_neighbors (K, sub_topics_embeds, sub_contents_embeds, topic_id_embeds, content_ids_embeds, df_corr, langs, langs_limit)

print ('\n All finished!\n')
print ('\n*** Predictions ***\n')
display(preds)
print ('\n*** Score ***\n')
display(fscore)


* Getting 'topics_breadcrumbs' dataframe...

Creating breadcrumbs...
Creating dataframe from breadcrumbs 'df_topics_breadcrumbs'...

* Calculating topics_features dataframe...


Creating and cleaning topic features...

Created 'topic_features'


,channel,category,level,language,has_content,sentences
t_00004da3a1b2,000cf7,source,four,bg,True,Откриването на резисторите.Изследване на матер...
t_00068291e9a4,8e286a,source,four,pt,True,Entradas e saídas de uma função.Entenda um pou...
t_00069b63a70a,6e3ba4,source,three,en,True,Transcripts..Flow Charts Logical Thinking Tr...
t_0006d41a73a8,000cf7,source,four,bg,True,Графики на експоненциални функции Алгебра 2 н...
t_4054df11a74e,6e3ba4,source,two,en,True,Flow Charts Logical Thinking .This lesson is ...



* Calculating content_features dataframe...


Creating and cleaning content features...

Created 'content_features'


,kind,language,license,sentences
id,,,,
c_0c6473c3480d,video,bg,CC BY-NC-SA,Чертане на показателни финкции.Сал чертае y 2...
c_1108dd0c7a5d,video,bg,NaN,Молив като резистор.Моливът причинява промяна ...
c_11a1dc0bfb99,document,en,CC BY-NC-SA,Flow Charts Logical Written Transcript of ...
c_1c57a1316568,video,bg,CC BY-NC-SA,Графики на показателни функции стар пример .С...
c_3695c5dc1df6,html5,en,CC BY-NC-SA,Additional Resources for Flow Charts Logical ...



* Calculating embeddings for both topics and contents text columns...


Getting embeddings for 1:pt from 3 languages


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


* Calculating k neighnors predictions...


Setting up KNN model...

 K value: 2

Fitting KNN model...

Getting content_ids for topics in 1:pt from 3 languages

Getting content_ids for topics in 2:bg from 3 languages

Getting content_ids for topics in 3:en from 3 languages

Calculating scores...

Topics to match content: 5

Correct predictions: 4

F2 Score: 0.32051279617850204



 All finished!


*** Predictions ***



,topic_id,content_ids
0,t_00068291e9a4,c_376c5a8eb028 c_0c6473c3480d
1,t_00004da3a1b2,c_1108dd0c7a5d c_5bc0e1e2cba0
2,t_0006d41a73a8,c_1c57a1316568 c_b972646631cb
3,t_00069b63a70a,c_1108dd0c7a5d c_0c6473c3480d
4,t_4054df11a74e,c_1108dd0c7a5d c_0c6473c3480d



*** Score ***



,topic_id,content_ids_pred,content_ids_actual,correct_pred,precision,recall,f0.5,f1,f2
0,t_00068291e9a4,"[c_376c5a8eb028, c_0c6473c3480d]","[c_639ea2ef9c95, c_89ce9367be10, c_ac1672cdcd2...",0,0.0,0.0,0.000000,0.000000,0.000000
1,t_00004da3a1b2,"[c_1108dd0c7a5d, c_5bc0e1e2cba0]","[c_1108dd0c7a5d, c_376c5a8eb028, c_5bc0e1e2cba...",2,0.5,1.0,0.555555,0.666667,0.833333
2,t_0006d41a73a8,"[c_1c57a1316568, c_b972646631cb]","[c_0c6473c3480d, c_1c57a1316568, c_5e375cf14c4...",2,0.4,1.0,0.454545,0.571429,0.769231
3,t_00069b63a70a,"[c_1108dd0c7a5d, c_0c6473c3480d]",[c_11a1dc0bfb99],0,0.0,0.0,0.000000,0.000000,0.000000
4,t_4054df11a74e,"[c_1108dd0c7a5d, c_0c6473c3480d]","[c_3695c5dc1df6, c_f2d184a98231]",0,0.0,0.0,0.000000,0.000000,0.000000


In [27]:
#import pickle
#pickle.dump(model, open(f'ST-KNN-{k[1]}', 'wb'))
preds.to_csv('submission.csv', index=False)